# Visualize pymoo result
Load the pymoo result object saved with pickle and visualize it.

ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

### Load nsga3 results for all topos

In [2]:
with open('res_bc_Cogent_nsga3.pkl','rb') as f_Cogent:
    res_4_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_nsga3.pkl','rb') as f_UsCarrier:
    res_4_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_nsga3.pkl','rb') as f_HiberniaGlobal:
    res_4_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_nsga3.pkl','rb') as f_Colt:
    res_4_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_nsga3.pkl','rb') as f_Funet:
    res_4_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_nsga3.pkl','rb') as f_Abvt:
    res_4_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_nsga3.pkl','rb') as f_Intellifiber:
    res_4_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_nsga3.pkl','rb') as f_TataNld:
    res_4_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_nsga3.pkl','rb') as f_Internode:
    res_4_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_nsga3.pkl','rb') as f_Missouri:
    res_4_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_nsga3.pkl','rb') as f_Ion:
    res_4_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_nsga3.pkl','rb') as f_Palmetto:
    res_4_Palmetto = pickle.load(f_Palmetto)

## Hypervolume
1. Store values

In [ ]:

# nsga3
F4_Cogent=res_4_Cogent.F
F4_UsCarrier=res_4_UsCarrier.F
F4_HiberniaGlobal=res_4_HiberniaGlobal.F
F4_Colt=res_4_Colt.F
F4_Funet=res_4_Funet.F
F4_Abvt=res_4_Abvt.F
F4_Intellifiber=res_4_Intellifiber.F
F4_TataNld=res_4_TataNld.F
F4_Internode=res_4_Internode.F
F4_Missouri=res_4_Missouri.F
F4_Ion=res_4_Ion.F
F4_Palmetto=res_4_Palmetto.F

# # Nadir Point from 9 Algorithms: 
# ref_point = [1.81000000e+02 6.20000000e+01 3.92281875e+03 1.70537209e+00]
# # Nadir Point from 4 Algorithms: 
# ref_point = [160.  77.  inf  inf]
# # Nadir Point from all (13) Algorithms: 
# ref_point = [181.  77.  inf  inf]

ref_point = [181, 77, 3922.81875, 1.70537209]

hist_F4_Cogent = []
hist_F4_UsCarrier = []
hist_F4_HiberniaGlobal = []
hist_F4_Colt = []
hist_F4_Funet = []
hist_F4_Abvt = []
hist_F4_Intellifiber = []
hist_F4_TataNld = []
hist_F4_Internode = []
hist_F4_Missouri = []
hist_F4_Ion = []
hist_F4_Palmetto = []


for algo4_Cogent in res_4_Cogent.history:
    opt4_Cogent = algo4_Cogent.opt
    feas4_Cogent = np.where(opt4_Cogent.get("feasible"))[0]
    hist_F4_Cogent.append(opt4_Cogent.get("F")[feas4_Cogent])
for algo4_UsCarrier in res_4_UsCarrier.history:
    opt4_UsCarrier = algo4_UsCarrier.opt
    feas4_UsCarrier = np.where(opt4_UsCarrier.get("feasible"))[0]
    hist_F4_UsCarrier.append(opt4_UsCarrier.get("F")[feas4_UsCarrier])
for algo4_HiberniaGlobal in res_4_HiberniaGlobal.history:
    opt4_HiberniaGlobal = algo4_HiberniaGlobal.opt
    feas4_HiberniaGlobal = np.where(opt4_HiberniaGlobal.get("feasible"))[0]
    hist_F4_HiberniaGlobal.append(opt4_HiberniaGlobal.get("F")[feas4_HiberniaGlobal])
for algo4_Colt in res_4_Colt.history:
    opt4_Colt = algo4_Colt.opt
    feas4_Colt = np.where(opt4_Colt.get("feasible"))[0]
    hist_F4_Colt.append(opt4_Colt.get("F")[feas4_Colt])
for algo4_Funet in res_4_Funet.history:
    opt4_Funet = algo4_Funet.opt
    feas4_Funet = np.where(opt4_Funet.get("feasible"))[0]
    hist_F4_Funet.append(opt4_Funet.get("F")[feas4_Funet])
for algo4_Abvt in res_4_Abvt.history:
    opt4_Abvt = algo4_Abvt.opt
    feas4_Abvt = np.where(opt4_Abvt.get("feasible"))[0]
    hist_F4_Abvt.append(opt4_Abvt.get("F")[feas4_Abvt])
for algo4_Intellifiber in res_4_Intellifiber.history:
    opt4_Intellifiber = algo4_Intellifiber.opt
    feas4_Intellifiber = np.where(opt4_Intellifiber.get("feasible"))[0]
    hist_F4_Intellifiber.append(opt4_Intellifiber.get("F")[feas4_Intellifiber])
for algo4_TataNld in res_4_TataNld.history:
    opt4_TataNld = algo4_TataNld.opt
    feas4_TataNld = np.where(opt4_TataNld.get("feasible"))[0]
    hist_F4_TataNld.append(opt4_TataNld.get("F")[feas4_TataNld])
for algo4_Internode in res_4_Internode.history:
    opt4_Internode = algo4_Internode.opt
    feas4_Internode= np.where(opt4_Internode.get("feasible"))[0]
    hist_F4_Internode.append(opt4_Internode.get("F")[feas4_Internode])
for algo4_Missouri in res_4_Missouri.history:
    opt4_Missouri = algo4_Missouri.opt
    feas4_Missouri = np.where(opt4_Missouri.get("feasible"))[0]
    hist_F4_Missouri.append(opt4_Missouri.get("F")[feas4_Missouri])
for algo4_Ion in res_4_Ion.history:
    opt4_Ion = algo4_Ion.opt
    feas4_Ion = np.where(opt4_Ion.get("feasible"))[0]
    hist_F4_Ion.append(opt4_Ion.get("F")[feas4_Ion])
for algo4_Palmetto in res_4_Palmetto.history:
    opt4_Palmetto = algo4_Palmetto.opt
    feas4_Palmetto = np.where(opt4_Palmetto.get("feasible"))[0]
    hist_F4_Palmetto.append(opt4_Palmetto.get("F")[feas4_Palmetto])

2. Calculate hypervolume and merge them (Mean value) for nsga3 algorithm

In [ ]:
from pymoo.indicators.hv import Hypervolume

metric = Hypervolume(ref_point= np.array(ref_point))

hv_4_Cogent = [metric.do(_F4_Cogent) for _F4_Cogent in hist_F4_Cogent]
hv_4_UsCarrier = [metric.do(_F4_UsCarrier) for _F4_UsCarrier in hist_F4_UsCarrier]
hv_4_HiberniaGlobal = [metric.do(_F4_HiberniaGlobal) for _F4_HiberniaGlobal in hist_F4_HiberniaGlobal]
hv_4_Colt = [metric.do(_F4_Colt) for _F4_Colt in hist_F4_Colt]
hv_4_Funet = [metric.do(_F4_Funet) for _F4_Funet in hist_F4_Funet]
hv_4_Abvt = [metric.do(_F4_Abvt) for _F4_Abvt in hist_F4_Abvt]
hv_4_Intellifiber = [metric.do(_F4_Intellifiber) for _F4_Intellifiber in hist_F4_Intellifiber]
hv_4_TataNld = [metric.do(_F4_TataNld) for _F4_TataNld in hist_F4_TataNld]
hv_4_Internode = [metric.do(_F4_Internode) for _F4_Internode in hist_F4_Internode]
hv_4_Missouri = [metric.do(_F4_Missouri) for _F4_Missouri in hist_F4_Missouri]
hv_4_Ion = [metric.do(_F4_Ion) for _F4_Ion in hist_F4_Ion]
hv_4_Palmetto = [metric.do(_F4_Palmetto) for _F4_Palmetto in hist_F4_Palmetto]

# nsga3
hv_4 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_4_Cogent, hv_4_UsCarrier, hv_4_HiberniaGlobal, hv_4_Colt, hv_4_Funet, hv_4_Abvt, hv_4_Intellifiber, hv_4_TataNld, hv_4_Internode, hv_4_Missouri, hv_4_Ion, hv_4_Palmetto)]

In [ ]:
print('The hypervolumn for Cogent:')
print(hv_4_Cogent)
print('The hypervolumn for UsCarrier:')
print(hv_4_UsCarrier)
print('The hypervolumn for HiberniaGlobal:')
print(hv_4_HiberniaGlobal)
print('The hypervolumn for Colt:')
print(hv_4_Colt)
print('The hypervolumn for Funet:')
print(hv_4_Funet)
print('The hypervolumn for Abvt:')
print(hv_4_Abvt)
print('The hypervolumn for Intellifiber:')
print(hv_4_Intellifiber)
print('The hypervolumn for TataNld:')
print(hv_4_TataNld)
print('The hypervolumn for Internode:')
print(hv_4_Internode)
print('The hypervolumn for Missouri:')
print(hv_4_Missouri)
print('The hypervolumn for Ion:')
print(hv_4_Ion)
print('The hypervolumn for Palmetto:')
print(hv_4_Palmetto)

# Show all hv for each topo
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_4_Cogent)+1)), hv_4_Cogent,  color='black', label='Cogent')
plt.plot(list(range(1, len(hv_4_UsCarrier)+1)), hv_4_UsCarrier,  color='blue', label='UsCarrier')
plt.plot(list(range(1, len(hv_4_HiberniaGlobal)+1)), hv_4_HiberniaGlobal,  color='red', label='HiberniaGlobal')
plt.plot(list(range(1, len(hv_4_Colt)+1)), hv_4_Colt,  color='orange', label='Colt')
plt.plot(list(range(1, len(hv_4_Funet)+1)), hv_4_Funet,  color=(0, 0, 1, 0.5), label='Funet')
plt.plot(list(range(1, len(hv_4_Abvt)+1)), hv_4_Abvt,  color='gray', label='Abvt')
plt.plot(list(range(1, len(hv_4_Intellifiber)+1)), hv_4_Intellifiber,  color='purple', label='Intellifiber')
plt.plot(list(range(1, len(hv_4_TataNld)+1)), hv_4_TataNld,  color='green', label='TataNld')
plt.plot(list(range(1, len(hv_4_Internode)+1)), hv_4_Internode,  color='#33FF57', label='Internode')
plt.plot(list(range(1, len(hv_4_Missouri)+1)), hv_4_Missouri,  color='brown', label='Missouri')
plt.plot(list(range(1, len(hv_4_Ion)+1)), hv_4_Ion,  color='pink', label='Ion')
plt.plot(list(range(1, len(hv_4_Palmetto)+1)), hv_4_Palmetto,  color='olive', label='Palmetto')
plt.title("Convergence")
plt.xlabel("Topos")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

In [ ]:
print('The aveage of hypervolumn for nsga3:')
print(hv_4)

3. Draw mean hypervolume for nsga3

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations") # brown,pink,#5733FF,olive,cyan,#FF5733,#33FF57
plt.ylabel("Hypervolume")
plt.legend()
plt.show()